In [3]:
# Start up the BQL service
import bql
import pandas as pd
import numpy as np
import bqplot as bqp
from ipydatagrid import DataGrid,TextRenderer

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth',None)

bq = bql.Service()   # <-- create BQL service object

In [4]:
d=pd.read_csv('dataframe_first.csv)')

FileNotFoundError: [Errno 2] No such file or directory: 'dataframe_first.csv)'

In [ ]:
region_country_mapping_df=pd.read_csv('region_country_mapping.csv')
master_region_list=list(region_country_mapping_df.REGION.unique())
rating_score_mapping_df=pd.read_csv('rating_score_mapping.csv')

In [ ]:
def get_esg_score_for_isin(isin_list):
    input_isin = [i.strip()+' Corp' for i in isin_list]
    security = bq.univ.esgTicker(input_isin)
    esg_score = bq.func.value(bq.data.esg_disclosure_score(fpt='a').fillna(limit='1',on='period')['value'],universe=security,mapby='lineage')


    req = bql.Request(input_isin, {
        'ESG Disclosure Score':esg_score,
    })
    res = bq.execute(req)
    df=res[0].df().reset_index()
    df['ID_ISIN']=[i.split()[0] for i in df['ID']]
    df=df.rename(columns={"ORIG_IDS:0":"ESG Ticker"})
    df=df[['ID_ISIN','ESG Ticker','ESG Disclosure Score']]
    return df

In [ ]:
def get_bonds_based_on_ticker(input_ticker = 'AAPL'):
    # Get all Bonds for the ticker
    active_bonds = bq.univ.bondsuniv('Active')
    bondsuniv = bq.univ.filter(active_bonds, bq.data.ticker() == input_ticker)

    YTW = bq.data.yield_()['value']
    name = bq.data.name()['value']
    maturity_years = bq.data.maturity_years()['value']
    ccy = bq.data.crncy()['value']
    rating = bq.data.bb_composite()['value']

    req = bql.Request(bondsuniv, {
        'Name':name,'YTW':YTW,'Time to Maturity':maturity_years,'Currency':ccy,'Rating':rating
    })
    res = bq.execute(req)
    data = bql.combined_df(res).dropna()
    data['Name']=data['Name']+' | '+data['Rating']+' | '+data['Currency']

    # Create data for chart
    dataframe = data.copy()

    # Create Scales
    scale_x = bqp.LinearScale()
    scale_y = bqp.LinearScale()

    # Create Mark
    tooltip = bqp.Tooltip(fields=['x', 'y','name'],
                          labels=['Time to Maturity', 'YTW','Name'],
                          formats=['.3f', '.3f'])
    mark_scatter = bqp.Scatter(x=dataframe['Time to Maturity'],
                               y=dataframe['YTW'],
                               names=dataframe['Name'], 
                                display_names=False,
                               tooltip=tooltip,
                               scales={'x': scale_x, 'y': scale_y})

    # Create Axes
    axis_x = bqp.Axis(scale=scale_x, label='Time to Maturity')
    axis_y = bqp.Axis(scale=scale_y, orientation='vertical', label='YTW')

    # Create Figure
    figure = bqp.Figure(marks=[mark_scatter],
                        axes=[axis_x, axis_y],
                        title=f"Yields for {input_ticker} Bonds",
                        title_style={'font-size': '22px'},
                        padding_x=0.05,
                        padding_y=0.05,
                        layout={'width': '100%', 'height': '500px'})

    # Display the figure
    return figure

In [ ]:
def get_bonds_based_on_ticker_and_maturity(input_ticker = 'AAPL',input_maturity_mm_dd_yyyy = '08/20/2060'):
    input_maturity_mm_dd_yyyy=pd.to_datetime(input_maturity_mm_dd_yyyy).date()
    active_bonds = bq.univ.bondsuniv('Active')
    bondsuniv = bq.univ.filter(active_bonds, bq.func.and_(bq.data.ticker() == input_ticker , 
                                                          bq.data.maturity() == input_maturity_mm_dd_yyyy))

    YTW = bq.data.yield_()['value']
    name = bq.data.name()['value']
    maturity_years = bq.data.maturity_years()['value']
    issue_date = bq.data.issue_dt()['value']
    sector = bq.data.bics_level_2_industry_group_name()['value']
    country = bq.data.cntry_of_risk()['value']
    rating = bq.data.bb_composite()['value']
    ccy = bq.data.crncy()['value']

    req = bql.Request(bondsuniv, {
        'Name':name,'YTW':YTW,'Time to Maturity':maturity_years,'Issue Date':issue_date,'Sector':sector,'Country of Risk':country,
        'Rating':rating,'Currency':ccy
    })
    res = bq.execute(req)
    data = bql.combined_df(res)
    data = data.dropna().sort_values(by="Issue Date").tail(1)
    data['Region']=region_country_mapping_df[region_country_mapping_df.COUNTRY_CODE==data['Country of Risk'].values[0]]['REGION'].values[0]
    data['Rating Score']=rating_score_mapping_df[rating_score_mapping_df.rating==data['Rating'].values[0]]['rating_score'].values[0]
    data['Name']=data['Name']+' | '+data['Rating']+' | '+data['Currency']
    
    # Get analytics for output bonds based on Region, Sector and Rating 
    bond_rating_score=data['Rating Score'].values[0]
    bond_region=data['Region'].values[0]
    bond_sector=data['Sector'].values[0]
    bond_ccy=data['Currency'].values[0]
    country_list_for_region=list(region_country_mapping_df[region_country_mapping_df.REGION==bond_region]['COUNTRY_CODE'].unique())
    ratings_list_required=list(rating_score_mapping_df[(rating_score_mapping_df['rating_score']>=bond_rating_score-1) &
                                                       (rating_score_mapping_df['rating_score']<=bond_rating_score+1)
                                                      ]['rating'].unique())
    active_bonds = bq.univ.bondsuniv('Active')
    bondsuniv = bq.univ.filter(active_bonds, bq.func.and_(bq.data.crncy()==bond_ccy,
                                                          bq.func.and_(bq.func.and_(bq.data.cntry_of_risk().in_(country_list_for_region) , 
                                                          bq.data.bb_composite().in_(ratings_list_required)),
                                                         bq.data.bics_level_2_industry_group_name() == bond_sector)))

    YTW = bq.data.yield_()['value']
    name = bq.data.name()['value']
    maturity_years = bq.data.maturity_years()['value']
    country = bq.data.cntry_of_risk()['value']
    rating=bq.data.bb_composite()['value']
    ccy = bq.data.crncy()['value']

    req = bql.Request(bondsuniv, {
        'Name':name,'YTW':YTW,'Time to Maturity':maturity_years,'Country of Risk':country,'Rating':rating,'Currency':ccy
    })
    res = bq.execute(req)
    datas = bql.combined_df(res)
    datas = datas.dropna()
    datas['Region']=bond_region
    datas['Sector']=bond_sector
    datas['Name']=datas['Name']+' | '+datas['Rating']+' | '+datas['Currency']

    # Create Chart
    dataframe = datas.copy()

    # Create Scales
    scale_x = bqp.LinearScale()
    scale_y = bqp.LinearScale()

    # Create Mark
    tooltip = bqp.Tooltip(fields=['x', 'y','name'],
                          labels=['Time to Maturity', 'YTW','Name'],
                          formats=['.3f', '.3f'])
    mark_scatter = [bqp.Scatter(x=dataframe['Time to Maturity'],
                               y=dataframe['YTW'],
                               names=dataframe['Name'], 
                                display_names=False,
                               tooltip=tooltip,
                               scales={'x': scale_x, 'y': scale_y}),
                    bqp.Scatter(x=data['Time to Maturity'],
                               y=data['YTW'],
                               names=dataframe['Name'], 
                                display_names=False,
                               tooltip=tooltip,
                                colors=['#FF5A00'],
                               scales={'x': scale_x, 'y': scale_y})
                   ]

    # Create Axes
    axis_x = bqp.Axis(scale=scale_x, label='Time to Maturity')
    axis_y = bqp.Axis(scale=scale_y, orientation='vertical', label='YTW')

    # Create Figure
    figure = bqp.Figure(marks=mark_scatter,
                        axes=[axis_x, axis_y],
                        title=f"Yields for {bond_sector} Bonds in {bond_region} region (Similar Rating)",
                        title_style={'font-size': '22px'},
                        padding_x=0.05,
                        padding_y=0.05,
                        layout={'width': '100%', 'height': '500px'})

    # Display the figure
    return figure

In [ ]:
def get_esg_alternatives(input_isin = 'US037833BA77'):
    #ESG Alternative
    # Get Region , Sector , Currency and rating for input bond
    
    security = input_isin.strip()+' Corp'

    bondsuniv = bq.univ.bonds(security)

    YTW = bq.data.yield_()['value']
    name = bq.data.name()['value']
    maturity_years = bq.data.maturity_years()['value']
    issue_date = bq.data.issue_dt()['value']
    sector = bq.data.bics_level_1_sector_name()['value'] #bics_level_2_industry_group_name
    country = bq.data.cntry_of_risk()['value']
    rating = bq.data.bb_composite()['value']
    duration = bq.data.duration()['value']
    ccy = bq.data.crncy()['value']


    req = bql.Request(bondsuniv, {
        'Name':name,'YTW':YTW,'Time to Maturity':maturity_years,'Issue Date':issue_date,'Sector':sector,'Country of Risk':country,
        'Rating':rating,'Duration':duration,'Currency':ccy
    })
    res = bq.execute(req)
    data = bql.combined_df(res)
    data = data.sort_values(by="Issue Date").tail(1)
    data['Region']=region_country_mapping_df[region_country_mapping_df.COUNTRY_CODE==data['Country of Risk'].values[0]]['REGION'].values[0]
    data['Rating Score']=rating_score_mapping_df[rating_score_mapping_df.rating==data['Rating'].values[0]]['rating_score'].values[0]
    data['ID_ISIN']=input_isin

    esg_df=get_esg_score_for_isin(isin_list=[input_isin])
    data=pd.merge(left=data.reset_index(),right=esg_df,how='left',left_on='ID_ISIN',right_on='ID_ISIN')

    # Get list of similar ESG Bonds, based on sector, ccy, region, rating+/-1, Duration+/-3

    bond_rating_score=data['Rating Score'].values[0]
    bond_region=data['Region'].values[0]
    bond_sector=data['Sector'].values[0]
    bond_ccy=data['Currency'].values[0]
    bond_duration=data['Duration'].values[0]
    bond_esg_ticker=data['ESG Ticker'].values[0]
    bond_esg_score=data['ESG Disclosure Score'].values[0]
    bond_id=data['ID'].values[0]
    country_list_for_region=list(region_country_mapping_df[region_country_mapping_df.REGION==bond_region]['COUNTRY_CODE'].unique())
    ratings_list_required=list(rating_score_mapping_df[(rating_score_mapping_df['rating_score']>=bond_rating_score-1) &
                                                       (rating_score_mapping_df['rating_score']<=bond_rating_score+1)
                                                      ]['rating'].unique())
    active_bonds = bq.univ.bondsuniv('Active')
    bondsuniv = bq.univ.filter(active_bonds, bq.func.and_(bq.data.duration().between(bond_duration-3, bond_duration+3),bq.func.and_(bq.data.crncy()==bond_ccy,
                                                          bq.func.and_(bq.func.and_(bq.data.cntry_of_risk().in_(country_list_for_region) , 
                                                          bq.data.bb_composite().in_(ratings_list_required)),
                                                         bq.data.bics_level_1_sector_name() == bond_sector))))

    YTW = bq.data.yield_()['value']
    name = bq.data.name()['value']
    maturity_years = bq.data.maturity_years()['value']
    country = bq.data.cntry_of_risk()['value']
    rating=bq.data.bb_composite()['value']
    duration = bq.data.duration()['value']
    isin = bq.data.id_isin()['value']


    req = bql.Request(bondsuniv, {
        'Name':name,'YTW':YTW,'Time to Maturity':maturity_years,'Country of Risk':country,'Rating':rating,'ID_ISIN':isin,
        'Duration':duration
    })
    res = bq.execute(req)
    datas = bql.combined_df(res)
    datas = datas.dropna()
    datas['Region']=bond_region
    datas['Sector']=bond_sector
    datas['Currency']=bond_ccy

    esg_df=get_esg_score_for_isin(isin_list=list(datas.ID_ISIN.unique()))
    datas=pd.merge(left=datas.reset_index(),right=esg_df,how='left',left_on='ID_ISIN',right_on='ID_ISIN')
    datas=datas[(datas['ESG Ticker']!=bond_esg_ticker) & datas['ESG Disclosure Score'].notnull()]
    datas=datas[datas['ESG Disclosure Score']>=bond_esg_score-5]
    
    # Find Price correlation
    base_bond=data.ID.values[0]
    universe = list(np.append(data.ID.values,datas.ID.values)) #First bond is the base bond
    date_range = bq.func.range('-1Y', '0D')
    closing_prices = bq.data.px_last(dates=date_range, per='D').dropna()
    req = bql.Request(universe, {'Price': closing_prices})
    res = bq.execute(req)
    df = res[0].df().reset_index().pivot(index='DATE', values='Price', columns='ID')     # <-- parse and pivot the dataframe in one line

    corr_df=pd.DataFrame(df.corr()[base_bond])
    corr_df.columns=['Price Correlation']
    esg_alternatives_df=pd.concat([data,pd.merge(left=datas,right=corr_df,how='left',left_on='ID',right_index=True)])
    column_list=['ID','Name','ID_ISIN','ESG Disclosure Score','Price Correlation','YTW','Duration','Rating',
                  'Region','Country of Risk','Currency','Sector']
    esg_alternatives_df=esg_alternatives_df[column_list].set_index('ID').round(3).T
    
    renderers = {bond_id:TextRenderer(background_color='green')}
    datagrid = DataGrid(dataframe=esg_alternatives_df,index_name="",layout={"height": "300px"},
                        base_column_size=130,
                        base_index_size=130,
                        renderers=renderers,
                        column_widths={'index':130, '#val':70},
                        )
    
    return datagrid

In [ ]:
# get_bonds_based_on_ticker(input_ticker = 'AAPL')

In [ ]:
# get_bonds_based_on_ticker_and_maturity(input_ticker = 'AAPL',input_maturity_mm_dd_yyyy = '08/20/2060')

In [ ]:
# get_esg_alternatives(input_isin = 'US037833BA77')

In [ ]:
import ipywidgets as widgets
from ipywidgets import interactive, interact_manual, interactive_output

In [ ]:
app_title = widgets.HTML('<h1>COMPARE-ABLE BONDS</h1>')


widget_ddn = widgets.Dropdown(options=["Get Bonds using Ticker",
                                      "Get Comparable Bonds using Ticker and Maturity",
                                      "Get ESG Alternatives for an ISIN"], description="Select Option", layout=widgets.Layout(width="40%"))

widget_text = widgets.Text(description='Enter Security' , layout=widgets.Layout(width="40%"))

widget_btn = widgets.Button(description="GO", layout=widgets.Layout(width="20%"), button_style='info', icon='fa-play')

widget_hbox = widgets.HBox(children=[widget_ddn, widget_text, widget_btn], layout=widgets.Layout(padding='10px 10px 10px 10px'))

output = widgets.Output()

app = widgets.VBox(children=[app_title, widget_hbox, output], layout=widgets.Layout(padding='10px 10px 10px 10px'))

In [ ]:
def on_ddn_changed(change):
    widget_text.value = ''
    
widget_ddn.observe(on_ddn_changed, names='value')

In [ ]:
def resolve_widget(option, value):
    if option == 'Get Bonds using Ticker':
        return get_bonds_based_on_ticker(input_ticker = value)
    
    elif option == 'Get Comparable Bonds using Ticker and Maturity':
        return get_bonds_based_on_ticker_and_maturity(input_ticker = value.split(' ')[0], input_maturity_mm_dd_yyyy =  value.split(' ')[1])
    
    elif option == 'Get ESG Alternatives for an ISIN':
        return get_esg_alternatives(input_isin = value)
    
    else:
        return 'Please Select Valid option'
    
@output.capture()
def go_button_clicked(b):
    output.clear_output()
    display(resolve_widget(option=widget_ddn.value, value=widget_text.value))
    
widget_btn.on_click(go_button_clicked)

In [ ]:
#app = interact_manual(resolve_widget, option=widget_ddn, value=widget_text)

In [ ]:
#app = interactive_output(resolve_widget, {'option':widget_ddn, 'value':widget_text})

In [ ]:
app